In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn import datasets
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [6]:
# Load the data
# iris.data = [(Sepal Length, Sepal Width, Petal Length, Petal Width)]
# 加载iris数据集，抽取花萼长度和花瓣宽度，分割每类的x_vals值和y_vals值
iris = datasets.load_iris()
x_vals = np.array([[x[0], x[3]] for x in iris.data])
y_vals = np.array([1 if y==0 else -1 for y in iris.target])


In [11]:
# Create graph
sess = tf.Session()


# Declare batch size
# 声明批量大小（偏向于更大批量大小）
batch_size = 150
lr_rate = 0.01
gamma_val = -25.0
 
# Initialize placeholders
x_data = tf.placeholder(shape=[None, 2], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)
prediction_grid = tf.placeholder(shape=[None, 2], dtype=tf.float32)
 
# Create variables for svm
b = tf.Variable(tf.random_normal(shape=[1,batch_size]))
 
# Gaussian (RBF) kernel
# 声明批量大小（偏向于更大批量大小）
gamma = tf.constant(gamma_val)
sq_dists = tf.multiply(2., tf.matmul(x_data, tf.transpose(x_data)))
my_kernel = tf.exp(tf.multiply(gamma, tf.abs(sq_dists)))
 
# Compute SVM Model
first_term = tf.reduce_sum(b)
b_vec_cross = tf.matmul(tf.transpose(b), b)
y_target_cross = tf.matmul(y_target, tf.transpose(y_target))
second_term = tf.reduce_sum(tf.multiply(my_kernel, tf.multiply(b_vec_cross, y_target_cross)))
loss = tf.negative(tf.subtract(first_term, second_term))
 
# Gaussian (RBF) prediction kernel
# 创建一个预测核函数
rA = tf.reshape(tf.reduce_sum(tf.square(x_data), 1),[-1,1])
rB = tf.reshape(tf.reduce_sum(tf.square(prediction_grid), 1),[-1,1])
pred_sq_dist = tf.add(tf.subtract(rA, tf.multiply(2., tf.matmul(x_data, tf.transpose(prediction_grid)))), tf.transpose(rB))
pred_kernel = tf.exp(tf.multiply(gamma, tf.abs(pred_sq_dist)))
 
# 声明一个准确度函数，其为正确分类的数据点的百分比
prediction_output = tf.matmul(tf.multiply(tf.transpose(y_target),b), pred_kernel)
prediction = tf.sign(prediction_output-tf.reduce_mean(prediction_output))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.squeeze(prediction), tf.squeeze(y_target)), tf.float32))
 
# Declare optimizer
my_opt = tf.train.GradientDescentOptimizer(lr_rate)
train_step = my_opt.minimize(loss)
 
# Initialize variables
init = tf.global_variables_initializer()
sess.run(init)
 
# Training loop
loss_vec = []
batch_accuracy = []
for i in range(300):
    rand_index = np.random.choice(len(x_vals), size=batch_size)
    rand_x = x_vals[rand_index]
    rand_y = np.transpose([y_vals[rand_index]])
    sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y})
 
    temp_loss = sess.run(loss, feed_dict={x_data: rand_x, y_target: rand_y})
    loss_vec.append(temp_loss)
 
    acc_temp = sess.run(accuracy, feed_dict={x_data: rand_x,
                       y_target: rand_y,
                       prediction_grid:rand_x})
    batch_accuracy.append(acc_temp)

In [12]:
batch_accuracy

[0.56666666,
 0.47333333,
 0.6066667,
 0.55333334,
 0.6333333,
 0.73333335,
 0.58666664,
 0.76,
 0.48666668,
 0.53333336,
 0.7133333,
 0.47333333,
 0.5133333,
 0.6066667,
 0.75333333,
 0.7733333,
 0.7266667,
 0.7,
 0.52666664,
 0.73333335,
 0.6,
 0.78,
 0.79333335,
 0.78,
 0.68666667,
 0.79333335,
 0.91333336,
 0.82666665,
 0.68666667,
 0.84,
 0.68666667,
 0.8666667,
 0.9266667,
 0.96,
 0.85333335,
 0.82666665,
 0.8,
 0.76,
 0.7866667,
 0.91333336,
 0.92,
 0.96666664,
 0.78,
 0.8466667,
 0.8666667,
 0.93333334,
 0.91333336,
 0.96,
 0.9266667,
 0.85333335,
 0.88,
 0.9266667,
 0.9266667,
 0.93333334,
 0.97333336,
 0.84,
 0.97333336,
 0.87333333,
 0.98,
 0.93333334,
 0.8933333,
 0.85333335,
 0.9,
 0.96666664,
 1.0,
 0.97333336,
 0.94666666,
 0.92,
 0.9066667,
 0.96666664,
 0.8933333,
 0.96,
 1.0,
 0.96666664,
 0.9866667,
 0.9266667,
 0.9533333,
 0.94666666,
 0.97333336,
 0.99333334,
 0.96,
 0.98,
 0.91333336,
 0.9533333,
 0.9266667,
 0.98,
 0.9866667,
 1.0,
 0.9533333,
 0.87333333,
 1.0,


In [13]:
loss_vec

[2.3025055,
 0.8025087,
 -0.6974889,
 -2.1974893,
 -3.6974878,
 -5.1974883,
 -6.697483,
 -8.197483,
 -9.697484,
 -11.197483,
 -12.69748,
 -14.197481,
 -15.697478,
 -17.19748,
 -18.697477,
 -20.197474,
 -21.697477,
 -23.197474,
 -24.697472,
 -26.19747,
 -27.697474,
 -29.197472,
 -30.69747,
 -32.197464,
 -33.697468,
 -35.197468,
 -36.697464,
 -38.19746,
 -39.69746,
 -41.197464,
 -42.697456,
 -44.19746,
 -45.69746,
 -47.19746,
 -48.697456,
 -50.197453,
 -51.697453,
 -53.197453,
 -54.697453,
 -56.19745,
 -57.69745,
 -59.197453,
 -60.697453,
 -62.197445,
 -63.697445,
 -65.19745,
 -66.69745,
 -68.19744,
 -69.69744,
 -71.19744,
 -72.69744,
 -74.19745,
 -75.69743,
 -77.19744,
 -78.69744,
 -80.19744,
 -81.69743,
 -83.19743,
 -84.69743,
 -86.19743,
 -87.69743,
 -89.197426,
 -90.697426,
 -92.197426,
 -93.69743,
 -95.19742,
 -96.69742,
 -98.197426,
 -99.69742,
 -101.19741,
 -102.69742,
 -104.197426,
 -105.69742,
 -107.19742,
 -108.69741,
 -110.19741,
 -111.69741,
 -113.19741,
 -114.69741,
 -116.19